# Generate 3D animation data of a single person from a single video from an unknown camera

## Prerequisites

In [2]:
%pip install move-ugc-python

  Using cached move_ugc_python-0.9.1-py3-none-any.whl (22 kB)
  Using cached gql-3.5.0-py2.py3-none-any.whl (74 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
  Using cached pydantic_settings-2.3.3-py3-none-any.whl (22 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl (202 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached anyio-4.4.0-py3-none-any.whl (86 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 875.6 kB/s eta 0:00:0031m1.6 MB/s eta 0:00:01
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
  Attempting uninstall: pydantic-core
    Found exi

## Imports

In [3]:
from datetime import datetime
import os
from pathlib import Path
import time
import requests
from move_ugc import MoveUgc

## API class

In [10]:
class MoveAI:
    def __init__(self, api_key, endpoint_url=None):
        self.api_key = api_key
        if endpoint_url is None:
            endpoint_url = 'https://api.move.ai/ugc/graphql'
        self.endpoint_url = endpoint_url
        self.client = MoveUgc(api_key=api_key, endpoint_url=endpoint_url)

    def create_files(self, video_path):
        video_file = self.client.files.create(file_type="mp4")

        with open(video_path, 'rb') as f:
            requests.put(video_file.presigned_url, data=f.read())

        return video_file.id

    def create_take(self, video_file_id,  metadata=None):
        if metadata is None:
            metadata = {"test": "test"}
        take = self.client.takes.create(
            video_file_id=video_file_id,
            metadata=metadata,
        )
        return take

    def get_take(self, take_id):
        take = self.client.takes.retrieve(id=take_id)
        return take

    def create_job(self, take_id):
        job = self.client.jobs.create(take_id=take_id, metadata={"test": "test_job"})
        return job

    def get_job(self, job_id, expand=False):
        # Get a job using the Move One Public API
        # Implement job retrieval logic using move_ugc_python SDK
        if expand is False:
            job = self.client.jobs.retrieve(id=job_id)
        else:
            job = self.client.jobs.retrieve(
                id=job_id, expand=["take", "outputs", "client"]
            )
        return job

    def download_outputs(self, job_id, output_dir, output_name):
        # make output dir if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        # get job
        job = self.get_job(job_id, expand=True)
        # for each output download the file in the output_dir
        output_paths = []
        for output in job.outputs:
            output_file_name = f"{output_name}{output.file.type}"
            output_path = os.path.join(output_dir, output_file_name)
            with open(output_path, 'wb') as f:
                response = requests.get(output.file.presigned_url)
                f.write(response.content)
            output_paths.append(output_path)
        return output_paths


## Run

In [25]:
# Initialize the MoveAI client, make sure to set the MOVE_API_KEY environment variable or pass it as an argument
client = MoveAI(os.environ.get('MOVE_API_KEY', '<Your Move AI API key if it is not set as environment variable>'))

if not client.api_key or client.api_key == '<Your Move AI API key if it is not set as environment variable>':
    raise ValueError('Please set the MOVE_API_KEY environment variable or pass it as an argument to MoveAI')

# Set the path to the video file and output directory
input_video_file = Path('data/input_videos/example.mp4')
output_dir = Path('data/output')

# Upload the video file to MoveAI
video_file_id = client.create_files(input_video_file)

# Create a take based on the id of the uploaded video and start the job
take = client.create_take(video_file_id)
job = client.create_job(take.id)

In [28]:
# Poll the job until it is finished
attempts = 0

while attempts < 100:
    job = client.get_job(job.id)
    update_str = f"[{datetime.now().isoformat()} | {attempts}] Job {job.id} is {job.state}"
    print(update_str)
    if job.state == 'FINISHED':
        outputs = client.download_outputs(job.id, output_dir, input_video_file.stem)
        print(f"Outputs downloaded to {output_dir}")
        print(f"Output files: {outputs}")
        break
    else:
        time.sleep(30)
        attempts += 1

[2024-06-19T19:45:10.148545 | 0] Job job-f61ef704-11b5-4444-8fa9-ca04aedda019 is FINISHED
Outputs downloaded to data/output


In [29]:
# play the generated video
from IPython.display import Video
Video(outputs[0])